# prototyping the indexing
## getting all the file paths

> TODO use generators / yield to prevent loading all into RAM? maybe.

In [1]:
import os

# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk("data/TREC8all/Adhoc"):
    path = root.split(os.sep)
    print((len(path) - 1) * '---', os.path.basename(root))
    # commented out to keep output short
    #for file in files:
    #    print(len(path) * '---', file)
import sys
sys.version
sys.version_info

------ Adhoc
--------- fr94
------------ 06
------------ 09
------------ 03
------------ 11
------------ 01
------------ 10
------------ 04
------------ 12
------------ 08
------------ 07
------------ 05
------------ 02
--------- ft
------------ ft941
------------ ft934
------------ ft931
------------ ft944
------------ ft924
------------ ft923
------------ ft943
------------ ft933
------------ ft942
------------ ft932
------------ ft921
------------ ft911
------------ ft922
--------- fbis
--------- latimes


sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)

In [2]:
import fs

In [3]:
documents_path = 'data/TREC8all/Adhoc'

In [4]:
fs.exists(documents_path)

True

In [5]:
import glob

In [6]:
list(fs.list())

['indexer.sh',
 'indexingPrototyping.ipynb',
 'Untitled Document',
 'searcher.sh',
 'LICSENSE',
 'travis.yml',
 'README.md',
 'assignment.pdf']

In [7]:
next(fs.find(pattern='*[!.]', path=documents_path))

'/home/jakob/advancedInformationRetrieval/project1/data/TREC8all/Adhoc/fr94/06/fr9406210'

In [8]:

for filenames in fs.find(documents_path):
    print(filenames)

## first read an sgml file and parse the tags

In [28]:
from bs4 import BeautifulSoup
path = 'data/TREC8all/Adhoc/fr94/03/fr9403020'
text_file = open(path, 'r').read()
soup = BeautifulSoup(text_file,'html.parser')
soup.find_all('docno')[:5]

[<docno> FR940302-0-00001 </docno>,
 <docno> FR940302-0-00002 </docno>,
 <docno> FR940302-0-00003 </docno>,
 <docno> FR940302-0-00004 </docno>,
 <docno> FR940302-0-00005 </docno>,
 <docno> FR940302-0-00006 </docno>,
 <docno> FR940302-0-00007 </docno>,
 <docno> FR940302-0-00008 </docno>,
 <docno> FR940302-0-00009 </docno>,
 <docno> FR940302-0-00010 </docno>,
 <docno> FR940302-0-00011 </docno>,
 <docno> FR940302-0-00012 </docno>,
 <docno> FR940302-0-00013 </docno>,
 <docno> FR940302-0-00014 </docno>,
 <docno> FR940302-0-00015 </docno>,
 <docno> FR940302-0-00016 </docno>,
 <docno> FR940302-0-00017 </docno>,
 <docno> FR940302-0-00018 </docno>,
 <docno> FR940302-0-00019 </docno>,
 <docno> FR940302-0-00020 </docno>,
 <docno> FR940302-0-00021 </docno>,
 <docno> FR940302-0-00022 </docno>,
 <docno> FR940302-0-00023 </docno>,
 <docno> FR940302-0-00024 </docno>,
 <docno> FR940302-0-00025 </docno>,
 <docno> FR940302-0-00026 </docno>,
 <docno> FR940302-0-00027 </docno>,
 <docno> FR940302-0-00028 </

## tokenization sample

In [10]:
import en_core_web_md
import spacy
from spacy.symbols import ORTH, LEMMA, POS

nlp = en_core_web_md.load()
doc = nlp(u'This is a sentence.')

contents of default pipeline. To customize use https://spacy.io/docs/usage/customizing-pipeline

In [11]:
for proc in nlp.pipeline:
    print(proc)

lemmatization


In [29]:
doc3 = nlp(u"this is spacy lemmatize testing. programming books are more better than others")

for token in doc3:
    print('token: ' + str(token) + ", lemma2: " + str(token.lemma_))
    print(token.lemma)
# print sentences
print("################")
for sent in doc3.sents:
    print(sent)

print("################")
    
for i, token in enumerate(doc3):
    print("original:", token.orth, token.orth_)
    print("lowercased:", token.lower, token.lower_)
    print("lemma:", token.lemma, token.lemma_)
    print("shape:", token.shape, token.shape_)
    print("prefix:", token.prefix, token.prefix_)
    print("suffix:", token.suffix, token.suffix_)
    print("log probability:", token.prob)
    print("Brown cluster id:", token.cluster)
    print("----------------------------------------")

token: this, lemma2: this
530
token: is, lemma2: be
522
token: spacy, lemma2: spacy
173815
token: lemmatize, lemma2: lemmatize
1484778
token: testing, lemma2: testing
2933
token: ., lemma2: .
453
token: programming, lemma2: programming
3441
token: books, lemma2: book
1045
token: are, lemma2: be
522
token: more, lemma2: more
563
token: better, lemma2: better
649
token: than, lemma2: than
589
token: others, lemma2: other
598
################
this is spacy lemmatize testing.
programming books are more better than others
################
original: 530 this
lowercased: 530 this
lemma: 530 this
shape: 53773 xxxx
prefix: 3631 t
suffix: 592 his
log probability: -5.36181640625
Brown cluster id: 63
----------------------------------------
original: 508 is
lowercased: 508 is
lemma: 522 be
shape: 21614 xx
prefix: 604 i
suffix: 508 is
log probability: -4.457748889923096
Brown cluster id: 762
----------------------------------------
original: 173815 spacy
lowercased: 173815 spacy
lemma: 173815 spacy

handling of file path i.e. not loosing the connection / information to the original file
http://stackoverflow.com/questions/42979472/inverted-index-in-python-with-spacy-as-tokenization-and-persistent-relation-to-o

but how to fill it?

some info is required to be stored: https://tuwel.tuwien.ac.at/mod/forum/discuss.php?d=85807

In [13]:
doc3.user_data

{}

In [14]:
for w in nlp(u'This is a sentence.'.upper()):
    print(w.text)
#    print(w.pos)
#    print(w.text_)

THIS
IS
A
SENTENCE
.


multi threaded tokenization using generator

In [15]:
texts = [u'One document.', u'...', u'Lots of documents']
# .pipe streams input, and produces streaming output
iter_texts = (texts[i % 3] for i in range(100000000))

In [16]:
# todo pick a way bigger batch size
for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_threads=4)):
    assert doc.is_parsed
    if i == 30:
        break
    #print(i)
    print(doc)

One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents
One document.
...
Lots of documents


In [17]:
[w.text for w in nlp(u'gimme that')]

['gimme', 'that']

## stemming

- https://github.com/explosion/spaCy/issues/327

Demonstrates basic stemming below. Now we just need to make sure that the tokens we obtained beforehand will play nicely

In [18]:
import snowballstemmer

In [19]:
stemmer = snowballstemmer.stemmer("english")
stemmer.stemWords("endosomes foo bar tomoorow I go to university".split())

['endosom', 'foo', 'bar', 'tomoorow', 'I', 'go', 'to', 'univers']

In [20]:
doc3 = nlp(u"this is spacy lemmatize testing. programming books are more better than others")

for token in doc3:
    print('token: ' + str(token)+ ", lemma2: " + str(token.lemma_) +
          " stemmer: " + stemmer.stemWords(token.text.split())[0])

token: this, lemma2: this stemmer: this
token: is, lemma2: be stemmer: is
token: spacy, lemma2: spacy stemmer: spaci
token: lemmatize, lemma2: lemmatize stemmer: lemmat
token: testing, lemma2: testing stemmer: test
token: ., lemma2: . stemmer: .
token: programming, lemma2: programming stemmer: program
token: books, lemma2: book stemmer: book
token: are, lemma2: be stemmer: are
token: more, lemma2: more stemmer: more
token: better, lemma2: better stemmer: better
token: than, lemma2: than stemmer: than
token: others, lemma2: other stemmer: other


## stopwords
called special words?

check out how to add a custom stopword. https://github.com/explosion/spaCy/issues/226

In [21]:
error#. do not yet run below code.

NameError: name 'error' is not defined

## indexing sample
- first step is to perform tokenization (with options from the commandline, + normalization)
- build the index

What to store:
- each document has an id number: `<num> Number: 401` which is translated into and id by `qrels.trec8.adhoc.parts1-5`

For the persistence of the index pickle could be used 

https://nlp.stanford.edu/IR-book/html/htmledition/single-pass-in-memory-indexing-1.html

In [22]:
#input = [word1, word2, ...]
#output = {word1: [pos1, pos2], word2: [pos2, pos434], ...}
def index_one_file(termlist):
	fileIndex = {}
	for index, word in enumerate(termlist):
		if word in fileIndex.keys():
			fileIndex[word].append(index)
		else:
			fileIndex[word] = [index]
	return fileIndex

In [23]:
#input = {filename: [word1, word2, ...], ...}
#res = {filename: {word: [pos1, pos2, ...]}, ...}
def make_indices(termlists):
	total = {}
	for filename in termlists.keys():
		total[filename] = index_one_file(termlists[filename])
	return total

dict reversal http://stackoverflow.com/questions/35945473/how-to-reverse-a-dictionary-in-python

In [24]:
#input = {filename: {word: [pos1, pos2, ...], ... }}
#res = {word: {filename: [pos1, pos2]}, ...}, ...}
def fullIndex(regdex):
	total_index = {}
	for filename in regdex.keys():
		for word in regdex[filename].keys():
			if word in total_index.keys():
				if filename in total_index[word].keys():
					total_index[word][filename].extend(regdex[filename][word][:])
				else:
					total_index[word][filename] = regdex[filename][word]
			else:
				total_index[word] = {filename: regdex[filename][word]}
	return total_index

second idea to kickstart inverted indexing http://stackoverflow.com/questions/28019543/inverted-index-given-a-list-of-document-tokens-using-python

In [25]:
from collections import defaultdict
def create_index (data):
    index = defaultdict(list)
    for i, tokens in enumerate(data):
        for token in tokens:
            index[token].append(i)
    return index

index = create_index([['a', 'b'], ['a', 'c']])
print(index)
print(index.keys())
print(index['a'])
print(index['b'])

defaultdict(<class 'list'>, {'a': [0, 1], 'b': [0], 'c': [1]})
dict_keys(['a', 'b', 'c'])
[0, 1]
[0]


## punctuation
- addd additional punctiation characters https://nicschrading.com/project/Intro-to-NLP-with-spaCy/